In [1]:
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch import optim, Tensor
import torch.nn.functional as F
import scipy.sparse as sp
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from torch_geometric.utils import structured_negative_sampling
from torch_geometric.data import download_url, extract_zip
from torch_geometric.nn.conv.gcn_conv import gcn_norm
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.datasets import AmazonBook, MovieLens
from torch_geometric.transforms import Compose, ToDevice, ToUndirected
from torch_geometric.data import Data
from torch_geometric.typing import Adj
from torch_sparse import SparseTensor, matmul
from torch_geometric.utils import train_test_split_edges


In [2]:
class LightGCN(MessagePassing): 
    def __init__(self, num_users, num_items, embedding_dim=64, K=3, add_self_loops=False):
        """Initializes LightGCN Model
        Args:
            num_users (int): Number of users
            num_items (int): Number of items
            embedding_dim (int, optional): Dimensionality of embeddings. Defaults to 8.
            K (int, optional): Number of message passing layers. Defaults to 3.
            add_self_loops (bool, optional): Whether to add self loops for message passing. Defaults to False.
        """
        super().__init__()
        self.num_users, self.num_items = num_users, num_items
        self.embedding_dim, self.K = embedding_dim, K
        self.add_self_loops = add_self_loops

        self.users_emb = nn.Embedding(
            num_embeddings=self.num_users, embedding_dim=self.embedding_dim) # e_u^0
        self.items_emb = nn.Embedding(
            num_embeddings=self.num_items, embedding_dim=self.embedding_dim) # e_i^0

        nn.init.normal_(self.users_emb.weight, std=0.1)
        nn.init.normal_(self.items_emb.weight, std=0.1)

    def forward(self, edge_index: SparseTensor):
        """Forward propagation of LightGCN Model.
        Args:
            edge_index (SparseTensor): adjacency matrix
        Returns:
            tuple (Tensor): e_u_k, e_u_0, e_i_k, e_i_0
        """
        edge_index_norm = self._compute_normalized_adjacency(edge_index)
        emb_0 = torch.cat([self.users_emb.weight, self.items_emb.weight]) # E^0
        embs = [emb_0]
        emb_k = emb_0
        for i in range(self.K):
            emb_k = self.propagate(edge_index_norm, x=emb_k)
            embs.append(emb_k)
        embs = torch.stack(embs, dim=1)
        emb_final = torch.mean(embs, dim=1) 
        users_emb_final, items_emb_final = torch.split(
            emb_final, [self.num_users, self.num_items]) 
        return users_emb_final, self.users_emb.weight, items_emb_final, self.items_emb.weight
    def _compute_normalized_adjacency(self, edge_index: SparseTensor) -> SparseTensor:
        """Compute normalized adjacency matrix A' = D^(-1/2) * A * D^(-1/2)"""
        edge_index_norm = gcn_norm(
            edge_index,
            add_self_loops=self.add_self_loops
        ) 
        return edge_index_norm
    def message(self, x_j: Tensor) -> Tensor:
        return x_j
    def message_and_aggregate(self, adj_t: SparseTensor, x: Tensor) -> Tensor:
        return matmul(adj_t, x)  


In [3]:
def bpr_loss(users_emb_final, users_emb_0, pos_items_emb_final, pos_items_emb_0, neg_items_emb_final, neg_items_emb_0, lambda_val):
    """Bayesian Personalized Ranking Loss as described in https://arxiv.org/abs/1205.2618
    Args:
        users_emb_final (torch.Tensor): e_u_k
        users_emb_0 (torch.Tensor): e_u_0
        pos_items_emb_final (torch.Tensor): positive e_i_k
        pos_items_emb_0 (torch.Tensor): positive e_i_0
        neg_items_emb_final (torch.Tensor): negative e_i_k
        neg_items_emb_0 (torch.Tensor): negative e_i_0
        lambda_val (float): lambda value for regularization loss term

    Returns:
        torch.Tensor: scalar bpr loss value
    """
    reg_loss = lambda_val * (users_emb_0.norm(2).pow(2) +
                             pos_items_emb_0.norm(2).pow(2) +
                             neg_items_emb_0.norm(2).pow(2)) 

    pos_scores = torch.mul(users_emb_final, pos_items_emb_final)
    pos_scores = torch.sum(pos_scores, dim=-1) 
    neg_scores = torch.mul(users_emb_final, neg_items_emb_final)
    neg_scores = torch.sum(neg_scores, dim=-1) 
    loss = -torch.mean(torch.nn.functional.softplus(pos_scores - neg_scores)) + reg_loss

    return loss

In [13]:
def sample_batch(batch_size, edge_index):
 # returns a tuple of 3 tensors. Tensor 1 -> user, Tensor 2 -> positive interactions, Tensor3-> Neg interactions
   print("Original edge_index:")
   print(edge_index)
   print(f"Edge index shape: {edge_index.shape}")
   edges = structured_negative_sampling(edge_index)
   edges = torch.stack(edges, dim=0)
   print("\nAfter negative sampling (first 5 samples):")
   print("Format: [user_indices, pos_item_indices, neg_item_indices]")
   print(edges[:, :5])  # Show first 5 samples
   indices = random.choices(
        [i for i in range(edges[0].shape[0])], k=batch_size)
   batch = edges[:, indices]
   user_indices, pos_item_indices, neg_item_indices = batch[0], batch[1], batch[2]
   return user_indices, pos_item_indices, neg_item_indices


In [5]:
def get_ground_truth(edge_index):
    """Generates dictionary of positive items for each user efficiently.
    Args:
        edge_index (torch.Tensor): 2 by N list of edges 
    Returns:
        dict: dictionary of positive items for each user
    """
    user_pos_items = {user.item(): [] for user in edge_index[0].unique()}
    for user, item in zip(edge_index[0], edge_index[1]):
        user_pos_items[user.item()].append(item.item())

    return user_pos_items

In [6]:
def metrics(groundTruth, r, k):
    num_correct_pred = torch.sum(r, dim=-1).float()
    user_num_liked = torch.tensor([len(groundTruth[i]) for i in range(len(groundTruth))], dtype=torch.float) 
    recall = torch.mean(num_correct_pred / user_num_liked)
    precision = torch.mean(num_correct_pred) / k
    return recall.item(), precision.item()

In [7]:
# # wrapper function to get evaluation metrics
# def get_metrics(model, edge_index, exclude_edge_indices, k):
#     """Computes the evaluation metrics: recall, precision, and ndcg @ k

#     Args:
#         model (LighGCN): lightgcn model
#         edge_index (torch.Tensor): 2 by N list of edges for split to evaluate
#         exclude_edge_indices ([type]): 2 by N list of edges for split to discount from evaluation
#         k (int): determines the top k items to compute metrics on

#     Returns:
#         tuple: recall @ k, precision @ k, ndcg @ k
#     """
#     user_embedding = model.users_emb.weight
#     item_embedding = model.items_emb.weight

#     rating = torch.matmul(user_embedding, item_embedding.T)

#     for exclude_edge_index in exclude_edge_indices:
#         user_pos_items = get_ground_truth(exclude_edge_index)
#         exclude_users = []
#         exclude_items = []
#         for user, items in user_pos_items.items():
#             exclude_users.extend([user] * len(items))
#             exclude_items.extend(items)

#         rating[exclude_users, exclude_items] = -(1 << 10)

#     _, top_K_items = torch.topk(rating, k=k)

#     users = edge_index[0].unique()

#     test_user_pos_items = get_ground_truth(edge_index)

#     test_user_pos_items_list = [
#         test_user_pos_items[user.item()] for user in users]

#     r = []
#     for user in users:
#         ground_truth_items = test_user_pos_items[user.item()]
#         label = list(map(lambda x: x in ground_truth_items, top_K_items[user]))
#         r.append(label)
#     r = torch.Tensor(np.array(r).astype('float'))

#     recall, precision = metrics(test_user_pos_items_list, r, k)
#     # ndcg = 

#     return recall, precision, 0

In [8]:
# def evaluation(model, edge_index, sparse_edge_index, exclude_edge_indices, k, lambda_val):
#     """Evaluates model loss and metrics including recall, precision, ndcg @ k

#     Args:
#         model (LighGCN): lightgcn model
#         edge_index (torch.Tensor): 2 by N list of edges for split to evaluate
#         sparse_edge_index (sparseTensor): sparse adjacency matrix for split to evaluate
#         exclude_edge_indices ([type]): 2 by N list of edges for split to discount from evaluation
#         k (int): determines the top k items to compute metrics on
#         lambda_val (float): determines lambda for bpr loss

#     Returns:
#         tuple: bpr loss, recall @ k, precision @ k, ndcg @ k
#     """
#     users_emb_final, users_emb_0, items_emb_final, items_emb_0 = model.forward(
#         sparse_edge_index)
#     edges = structured_negative_sampling(
#         edge_index, contains_neg_self_loops=False)
#     user_indices, pos_item_indices, neg_item_indices = edges[0], edges[1], edges[2]
#     users_emb_final, users_emb_0 = users_emb_final[user_indices], users_emb_0[user_indices]
#     pos_items_emb_final, pos_items_emb_0 = items_emb_final[
#         pos_item_indices], items_emb_0[pos_item_indices]
#     neg_items_emb_final, neg_items_emb_0 = items_emb_final[
#         neg_item_indices], items_emb_0[neg_item_indices]

#     loss = bpr_loss(users_emb_final, users_emb_0, pos_items_emb_final, pos_items_emb_0,
#                     neg_items_emb_final, neg_items_emb_0, lambda_val).item()

#     recall, precision, ndcg = get_metrics(
#         model, edge_index, exclude_edge_indices, k)

#     return loss, recall, precision, ndcg

In [9]:
# ITERATIONS = 10000
# BATCH_SIZE = 1024
# LR = 1e-3
# ITERS_PER_EVAL = 200
# ITERS_PER_LR_DECAY = 200
# K = 20
# LAMBDA = 1e-6

In [10]:
# saved_tensors = torch.load('sparse_tensors.pt')
# train_sparse = saved_tensors['train_sparse']
# val_sparse = saved_tensors['val_sparse']
# test_sparse = saved_tensors['test_sparse']

# sparse_sizes = train_sparse.sparse_sizes()
# num_users = num_items = sparse_sizes[0] // 2
# print(f"Number of users/items: {num_users}")

# train_edge_index = torch.stack([train_sparse.storage.row(), train_sparse.storage.col()])
# val_edge_index = torch.stack([val_sparse.storage.row(), val_sparse.storage.col()])

# model = LightGCN(
#     num_users=num_users,
#     num_items=num_items,
#     embedding_dim=64,
#     K=3
# )

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print(f"Using device {device}")

# model = model.to(device)
# model.train()

# train_edge_index = train_edge_index.to(device)
# val_edge_index = val_edge_index.to(device)
# train_sparse = train_sparse.to(device)
# val_sparse = val_sparse.to(device)

# optimizer = optim.Adam(model.parameters(), lr=1e-4)  
# scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)

# print(f"Training setup complete. Ready to train on {device}.")

In [11]:
# train_losses = []
# val_losses = []

# for iter in range(ITERATIONS):
#    users_emb_final, users_emb_0, items_emb_final, items_emb_0 = model.forward(
#        train_sparse)  

#    user_indices, pos_item_indices, neg_item_indices = sample_batch(
#        BATCH_SIZE, train_edge_index)
#    user_indices, pos_item_indices, neg_item_indices = user_indices.to(
#        device), pos_item_indices.to(device), neg_item_indices.to(device)
#    users_emb_final, users_emb_0 = users_emb_final[user_indices], users_emb_0[user_indices]
#    pos_items_emb_final, pos_items_emb_0 = items_emb_final[
#        pos_item_indices], items_emb_0[pos_item_indices]
#    neg_items_emb_final, neg_items_emb_0 = items_emb_final[
#        neg_item_indices], items_emb_0[neg_item_indices]

#    train_loss = bpr_loss(users_emb_final, users_emb_0, pos_items_emb_final,
#                          pos_items_emb_0, neg_items_emb_final, neg_items_emb_0, LAMBDA)

#    optimizer.zero_grad()
#    train_loss.backward()
#    optimizer.step()

#    if iter % ITERS_PER_EVAL == 0:
#        model.eval()
#        val_loss, recall, precision, ndcg = evaluation(
#            model, val_edge_index, val_sparse, [train_edge_index], K, LAMBDA)  
#        print(f"[Iteration {iter}/{ITERATIONS}] train_loss: {round(train_loss.item(), 5)}, val_loss: {round(val_loss, 5)}, val_recall@{K}: {round(recall, 5)}, val_precision@{K}: {round(precision, 5)}, val_ndcg@{K}: {round(ndcg, 5)}")
#        train_losses.append(train_loss.item())
#        val_losses.append(val_loss)
#        model.train()

#    if iter % ITERS_PER_LR_DECAY == 0 and iter != 0:
#        scheduler.step()